### Theory About Reproducibility
Reproducing or rather producing the quantized model is the deadliest pain the ass I have felt in a while. It was bad.

Keeping the technicalities about SELECT_OPS and Flex Delegate etc aside, just confirm your setup as follow. I have already talked about using virtual environment as kernel in diskloader.ipynb

Final setup
- OS: Ubuntu Server  
- Python: 3.12  
- TensorFlow: 2.18.0 (regular, CPU-only)  
  - Do NOT use `tensorflow[and-cuda]` (conversion fails).  
  - Regular `tensorflow==2.18.0` works fine with Python 3.12.
    ```bash
    pip uninstall -y tensorflow[and-cuda]
    pip install tensorflow==2.18.*
    ````

### Prerequisites (uncomment to run)

#### Installations

In [1]:
# Assuming virtual environment .tens with python 3.12 is being used
%pip uninstall -y tensorflow[and-cuda]
%pip install tensorflow==2.18.* tqdm

Found existing installation: tensorflow 2.18.1
Uninstalling tensorflow-2.18.1:
  Successfully uninstalled tensorflow-2.18.1
Note: you may need to restart the kernel to use updated packages.
  Using cached tensorflow-2.18.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
Using cached tensorflow-2.18.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.6 MB)
Note: you may need to restart the kernel to use updated packages.


#### Data Preprocessing
- No need to re-run if already ran in `diskloader.ipynb`

In [2]:
# # Step 1: Download QuickDraw dataset
# %python download_data.py

# # Step 2: Combine stroke data across different partitions (train, val, test).
# # The original data (e.g., airplanes.npz) is split into these partitions, 
# # which reduces the number of data points. We merge all partitions for a complete dataset.
# %python combine_strokes.py

# # Step 3: Convert each `.npy` and `.npz` file into individual sample files.
# # For example, `airplanes.npy` contains (N, 784), where N is the number of samples 
# # and each sample is a flattened 28x28 image (784 features). We split this into
# # individual `.npy` files like `airplanes/000001.npy`, `airplanes/000002.npy`, etc.
# # This allows us to load specific slices of data (e.g., airplanes[3012:33012]) 
# # without having to load the entire file, reducing memory usage and avoiding memory eviction issues.
# %python convert_to_individual_npys.py

#### Imports

In [3]:
import json
import math
import os
from glob import glob

import numpy as np
import tensorflow as tf
from tqdm import tqdm

2025-08-17 11:03:39.618919: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755428619.630671  112379 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755428619.634025  112379 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Converting the Model

In [4]:
model = tf.keras.models.load_model('/home/priyanka/doodle-vision/training/best_model_345_classes_30000_examples.keras')

W0000 00:00:1755428621.989813  112379 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable Flex ops (Select TensorFlow Ops)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]

# Optional optimizations
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert
tflite_model = converter.convert()

# Save the model
with open('best_model_345_classes_30000_examples.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpbmwcyls1/assets


INFO:tensorflow:Assets written to: /tmp/tmpbmwcyls1/assets


Saved artifact at '/tmp/tmpbmwcyls1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 130, 3), dtype=tf.float32, name='stroke_input'), TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='image_input')]
Output Type:
  TensorSpec(shape=(None, 345), dtype=tf.float32, name=None)
Captures:
  140073260805264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140073260801808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140073260807568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140073260806800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140073260807184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140073260808144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140073260808528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140073260807376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140073260255952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140073

W0000 00:00:1755428623.369812  112379 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1755428623.369833  112379 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1755428623.394882  112379 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


### Loading and Using the model

In [6]:
interpreter = tf.lite.Interpreter(model_path="best_model_345_classes_30000_examples.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

/home/priyanka/doodle-vision/.tens/lib/python3.12/site-packages/tensorflow/lite/python/interpreter.py:451: UserWarning:     Warning: Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite delegate for select TF ops.
W0000 00:00:1755428623.888249  112379 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO: TfLiteFlexDelegate delegate: 8 nodes delegated out of 61 nodes with 4 partitions.

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
print("Input details:")
for i, d in enumerate(input_details):
    print(f"\tInput {i}: name = {d['name']}, shape = {d['shape']}, dtype = {d['dtype']}")

print("Output details:")
for i, d in enumerate(output_details):
    print(f"\tInput {i}: name = {d['name']}, shape = {d['shape']}, dtype = {d['dtype']}")

Input details:
	Input 0: name = serving_default_image_input:0, shape = [ 1 28 28  1], dtype = <class 'numpy.float32'>
	Input 1: name = serving_default_stroke_input:0, shape = [  1 130   3], dtype = <class 'numpy.float32'>
Output details:
	Input 0: name = StatefulPartitionedCall_1:0, shape = [  1 345], dtype = <class 'numpy.float32'>


In [8]:
# Indices for inputs and outputs. Used when feeding the data to model later.
image_idx = [i for i, d in enumerate(input_details) if "image_input" in d["name"]][0]
stroke_idx = [i for i, d in enumerate(input_details) if "stroke_input" in d["name"]][0]
output_idx = 0  # only one output

### Using the same dataloader pipeline as used in training

### Global Variables

In [9]:
# Files
LABELS_JSON    = "/home/priyanka/doodle-vision/training/dataset/label_map.json"
PROCESSED_DATA_DIR = "/home/priyanka/doodle-vision/training/dataset/processed"

# Data props
NUM_CLASSES = 345

# Training
SAMPLES_PER_CLASS = 30_000
SPLIT_RATIOS = (0.8, 0.1, 0.1) # train, val, test
BATCH_SIZE = 512

In [10]:
with open(LABELS_JSON, 'r') as f:
    LABEL_MAP = json.load(f)
    
REV_LABEL_MAP = {v: k for k, v in LABEL_MAP.items()}

### Disk Data Loader for Original Data

In [11]:
# Using the individual + in-memory mode for tflite inference
images = []
strokes = []

# Always list of integers
labels = []

for cls, label in tqdm(LABEL_MAP.items(), desc="Building dataset"):
    image_files = sorted(
        glob(os.path.join(PROCESSED_DATA_DIR, "images", cls, "*.npy"))
    )
    stroke_files = sorted(
        glob(os.path.join(PROCESSED_DATA_DIR, "strokes", cls, "*.npy"))
    )

    N = min(len(image_files), len(stroke_files), SAMPLES_PER_CLASS)

    for i in range(N):
        image_path = image_files[i]
        stroke_path = stroke_files[i]

        images.append(np.load(image_path))
        strokes.append(np.load(stroke_path))
        labels.append(label)

print(f"[In-Memory Mode] Loaded {len(labels):,} samples.")

Building dataset: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 345/345 [19:58<00:00,  3.47s/it]

[In-Memory Mode] Loaded 10,350,000 samples.


In [12]:
def data_generator(images, strokes, labels):
    for image, stroke, label in zip(images, strokes, labels):
        yield (
            image,
            stroke,
            tf.one_hot(label, depth=NUM_CLASSES),
        )

In [13]:
def format_sample(img, stroke, label):
    return {"stroke_input": stroke, "image_input": img}, label

output_signature = (
    tf.TensorSpec(shape=(28, 28, 1), dtype=tf.float32),
    tf.TensorSpec(shape=(130, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(NUM_CLASSES,), dtype=tf.int32),
)

### Train, Val, and Test Splits

In [14]:
# Shuffle once and split
total = len(labels)
indices = np.arange(total)
np.random.seed(42)
np.random.shuffle(indices)

train_end = int(SPLIT_RATIOS[0] * total)
val_end = train_end + int(SPLIT_RATIOS[1] * total)

train_idx = indices[:train_end]
val_idx = indices[train_end:val_end]
test_idx = indices[val_end:]

# Train
train_images =  [images[i] for i in train_idx]
train_strokes = [strokes[i] for i in train_idx]
train_labels =  [labels[i] for i in train_idx]

# Val
val_images =  [images[i] for i in val_idx]
val_strokes = [strokes[i] for i in val_idx]
val_labels =  [labels[i] for i in val_idx]

# Test
test_images =  [images[i] for i in test_idx]
test_strokes = [strokes[i] for i in test_idx]
test_labels =  [labels[i] for i in test_idx]

In [15]:
def build_dataset(images, strokes, labels, is_shuffle=False):
    def gen():
        return data_generator(images, strokes, labels)

    ds = tf.data.Dataset.from_generator(
        gen,
        output_signature=output_signature
    )

    if is_shuffle:
        ds = ds.shuffle(BATCH_SIZE * 10)

    ds = ds.repeat()
    ds = ds.map(format_sample, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE)
    return ds.prefetch(tf.data.AUTOTUNE)

In [16]:
test_ds = build_dataset(test_images, test_strokes, test_labels).take(math.ceil(len(test_labels) / BATCH_SIZE))

### Running Inference on TFLite Model

In [ ]:
correct = 0
total = 0

# Create tqdm object
pbar = tqdm(test_ds, desc="Evaluating TFLite", dynamic_ncols=True)

for batch in pbar:
    # Unpack batch
    inputs, labels = batch
    images = inputs["image_input"].numpy()
    strokes = inputs["stroke_input"].numpy()
    labels = labels.numpy()  # one-hot vectors

    for i in range(len(images)):
        # Set input tensors
        interpreter.set_tensor(input_details[image_idx]['index'], images[i:i+1])
        interpreter.set_tensor(input_details[stroke_idx]['index'], strokes[i:i+1])

        # Run inference
        interpreter.invoke()

        # Get predictions
        preds = interpreter.get_tensor(output_details[output_idx]['index'])
        pred_class = np.argmax(preds, axis=1)[0]
        true_class = np.argmax(labels[i])

        if pred_class == true_class:
            correct += 1
        total += 1

    # update tqdm live with accuracy
    pbar.set_postfix(acc=f"{correct/total:.4f}")


Evaluating TFLite: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2022/2022 [2:21:26<00:00,  4.20s/it, acc=0.8916]


In [18]:
# Final accuracy
accuracy = correct / total
print(f"TFLite Test Accuracy: {accuracy:.4f} ({correct}/{total})")

TFLite Test Accuracy: 0.8916 (922998/1035264)
